In [1]:
import plotly.express as px

import numpy as np

In [100]:
from neo4j import GraphDatabase, basic_auth

from neo4j_process.requests import get_all_communes_with_coords


In [101]:
import os

import psycopg2

import pandas as pd

# POSTGRES Variables

HOST_POSTGRES = 'localhost' #"172.17.0.1"

POSTGRES_PORT = "5433"

POSTGRES_DATABASE = "postgres"

POSTGRES_TABLE = "insee_value"

POSTGRES_USER = "postgres"

POSTGRES_PASSWORD = "my_pass"

# NEO4J Variables

HOST_NEO4J = "bolt://localhost:7687"

PASSWORD_NEO4J = "passwordneo4j"


In [102]:
# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host= HOST_POSTGRES,
    port=POSTGRES_PORT,
    database=POSTGRES_DATABASE,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD)

conn.autocommit = True

# Création d'un curseur
cur = conn.cursor()




In [103]:
query = "SELECT * FROM insee_value; "

In [104]:
result = cur.execute(query)

data= cur.fetchall()

conn.commit()
conn.close()

In [105]:
df_value = pd.DataFrame(data, columns=["insee", "value"])

In [106]:

# --- NEO4J Connection --- #

driver = GraphDatabase.driver(
  HOST_NEO4J,
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

In [107]:
with driver.session() as session:
       
    result = session.execute_write(get_all_communes_with_coords)

driver.close()

df_communes = pd.DataFrame(result)

In [108]:
df_communes

,insee,x,y
0,62053,2.143174,50.582893
1,80284,3.012390,49.717236
2,31290,0.610023,42.876728
3,28400,1.514000,48.094594
4,37220,0.099943,47.193507
...,...,...,...
34455,3170,3.092677,46.506657
34456,14394,-0.162877,49.015889
34457,29115,-3.895241,48.302289
34458,49308,-0.496845,47.370808


In [109]:
df_join_score = df_value.merge(df_communes, how="left", on="insee")

In [110]:
df_join_score

,insee,value,x,y
0,12044,71,2.878951,43.822949
1,17043,114,-0.618154,46.069645
2,3091,655,3.426865,46.261532
3,19163,802,1.738416,45.070132
4,74114,154,6.670308,46.201713
...,...,...,...,...
18089,60436,4,2.394022,49.591211
18090,63095,4,3.432198,45.944417
18091,31179,3,1.824386,43.567508
18092,57595,13,6.842011,48.830257


In [111]:
df_join_score["ln_value"] = df_join_score["value"].apply(lambda v : np.log(v))

In [112]:
fig = px.scatter_mapbox(df_join_score, lat="y", lon="x", mapbox_style="open-street-map", color_continuous_scale="reds", color="ln_value", hover_name="insee")
fig.show()

In [83]:
fig.write_html("./../data/score_2_visualisation_200k_draws.html")